In [34]:
import sys    # 시스템
import os     # 시스템

import pandas as pd    # 판다스 : 데이터분석 라이브러리
import numpy as np     # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup     # html 데이터 전처리
from selenium import webdriver    # 웹 브라우저 자동화
import time                       # 시간 지연
from tqdm import tqdm_notebook    # 진행상황 표시

In [35]:
### Step 1. 크롤링할 블로그 url 수집

driver=webdriver.Chrome('/Users/leesh970930/Desktop/chromedriver')


driver.get("http://www.naver.com")
time.sleep(2)

# 검색창에 '검색어' 검색
element = driver.find_element_by_id("query") #f12눌러서 확인
element.send_keys('직장인취미')
element.submit()
time.sleep(1)

In [36]:
# 'VIEW' 클릭
driver.find_element_by_link_text("VIEW").click()

# '블로그' 클릭
driver.find_element_by_link_text("블로그").click()
time.sleep(1)

# '옵션' 클릭
driver.find_element_by_link_text("옵션").click()
time.sleep(0.5)

# 검색옵션 확인
item_li = driver.find_elements_by_css_selector(".option .txt")

for i in range(0, len(item_li)):
    print(item_li[i].text)

# 검색기간 '3개월' 클릭
print(item_li[10].text)

item_li[10].click()

전체
블로그
카페
관련도순
최신순
전체
1시간
1일
1주
1개월
3개월
6개월
1년
직접입력옵션펼치기
3개월


In [37]:
# 스크롤 다운
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 99999999)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 10
i = 0
while i < n:
    scroll_down(driver)
    i = i+1

# 블로그 글 url들 수집
url_list = []
title_list = []

In [38]:
# URL_raw 크롤링 시작
article_raw = driver.find_elements_by_css_selector(".api_txt_lines.total_tit")
article_raw[0]

# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')   
    url_list.append(url)

time.sleep(1)
    
# 제목 크롤링 시작    
for article in article_raw:
    title = article.text
    title_list.append(title)

print("url개수: ", len(url_list))
print("title개수: ", len(title_list))

df = pd.DataFrame({'url':url_list, 'title':title_list})

url개수:  330
title개수:  330


In [39]:
### Step 2. 블로그 내용 크롤링

# URL이 저장된 리스트 불러오기
for url_load in url_list:
        print(url_load)
        
print("수집한 url 개수: ")
print(len(url_list))

https://blog.naver.com/onepiece0109?Redirect=Log&logNo=222576018957
https://blog.naver.com/ryouma77?Redirect=Log&logNo=222572805493
https://blog.naver.com/happylife_mi?Redirect=Log&logNo=222491886835
https://blog.naver.com/punytraveler?Redirect=Log&logNo=222497936363
https://blog.naver.com/marie81a?Redirect=Log&logNo=222565244063
https://blog.naver.com/aliesgrises?Redirect=Log&logNo=222564152637
https://blog.naver.com/vbbbb2001?Redirect=Log&logNo=222563326044
https://blog.naver.com/destiny6852?Redirect=Log&logNo=222535481934
https://blog.naver.com/cute0014?Redirect=Log&logNo=222544328807
https://blog.naver.com/limang2020?Redirect=Log&logNo=222572223869
https://blog.naver.com/jh_l0825?Redirect=Log&logNo=222498916895
https://blog.naver.com/dbtlsgus?Redirect=Log&logNo=222540913376
https://blog.naver.com/grimsooop?Redirect=Log&logNo=222557902812
https://blog.naver.com/yousuyoon?Redirect=Log&logNo=222547054637
https://blog.naver.com/fayefaye79?Redirect=Log&logNo=222574499695
https://blog.na

In [40]:
dict = {}    # 전체 크롤링 데이터를 담을 그릇 선언
number = len(url_list)    # 수집할 글 갯수


# 수집한 url 돌면서 데이터 수집
for i in tqdm_notebook(range(0, number)):
    # 글 띄우기
    url = url_list[i]
    driver = webdriver.Chrome('/Users/leesh970930/Desktop/chromedriver')
    driver.get(url)
    
    
    # 크롤링
    try : 
        # 글의 iframe 접근
        driver.switch_to.frame("mainFrame")

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

        # 제목 크롤링
        overlays = ".se-module.se-module-text.se-title-text"                        
        tit = driver.find_element_by_css_selector(overlays)
        title = tit.text

        # 글쓴이 크롤링
        overlays = ".nick"                                 
        nick = driver.find_element_by_css_selector(overlays)
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"    
        date = driver.find_element_by_css_selector(overlays)
        datetime = date.text
        
        # 해쉬태그 크롤링
        #overlays = "_param(false|false|4)" 
        #hashtag = driver.find_element_by_css_selector(overlays)
        #hashtag = hashtag.text

    
        # 크롤링한 글은 target_info라는 딕셔너리에 담음
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        #target_info['hashtag'] = hashtag

        # 각각의 글은 dict라는 딕셔너리에 담음
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링 성공하면 글 제목을 출력
        print(i, title)

        # 글 하나 크롤링 후 크롬 창 닫기
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동
    except:
        driver.close()
        time.sleep(1)
        continue
        


<ipython-input-40-fdbdd0b3fac5>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(0, number)):


0 대구비누공방 아낙 anak 직장인취미 나만의 비누
1 직장인 취미 추천 칼림바 어쿠스틱 맘에들어
2 직장인 취미기록[ 한국무용/발레/요가 ]
3 [직장인 취미찾기 프로젝트] 나만의 향수 만들기 신촌 원데이 클래스
4 직장인 취미 공방 안 가고 집에서 했어요
5 있지 로꼬 (Itzy - Loco) 직장인 취미 커버 댄스 영상 촬영 현장 스케치
6 직장인 취미로 홈베이킹 결과물 1탄
7 직장인 취미 생활 한 곳에서 끝
8 직장인 취미 생활, 집에서 할수있는 취미 추천 칼림바 네코즈 17SDP 
9 부산서면직장인취미미술 그림그린화실 서면원데이클래스
10 직장인 취미기록[드림스카이 색칠공부도안/컬러링북]
11 [위드피아노 목동점] 직장인취미 추천 피아노 한곡 완주하기!
13 [오일파스텔] 직장인의 취미 :-) 오일파스텔로 아기그림그리기
14 직장인 취미생활 추천 핸드에이블 손바느질 클래스
15 직장인취미 지식플랫폼 [퍼블리] 구독 1개월 후기!
16 [직장인 취미찾기 프로젝트] 처음해본 원목 작업 나만의 나무 식기류 만들기(feat.우드카빙)
17 직장인 퇴근 후 취미생활추천, 조맹 크로키 클래스101후기
18 직장인 취미생활 보석십자수에 푹 빠졌어요
19 부산직장인취미미술은 그림그린화실에서
20 해바라기 꽃그림 그리기_원데이클래스(직장인 취미미술)
21 직장인 취미찾기 연필 인물화 수업 #2 (눈 그리기)
22 [아이러브페인팅] 디즈니 보석십자수 : 집에서 즐기는 직장인취미!
23 정자역 에이지드로잉 원데이클래스 후기 | 직장인취미 드로잉취미 즐기기 아크릴 유화 페인팅 나이프화
24 [스트릿댄스학원] 스우파로 아직 과몰입 중인 직장인 분들을 위한 취미 성인 댄스학원 초보 걸스힙합 클래스 추천! | 강남댄스학원 댄스조아
25 사당역/이수역 1:1 피아노 개인레슨 (취미/직장인/성인/입시) '도로시피아노스튜디오'
26 직장인 취미생활추천! 힐링이 되는 금속공예로 상상의 날개를 펴보세요!
27 햇살 가득한 감성 유화 그리기 후기, 아트클래스 자그마치 직장인취미미술클래

211 [신세계 센텀 문화센터] 직장인 취미미술, 펜 드로잉 신청했어요
212 [직장인 취미 추천] 누구든지 쉽게 연주할 수 있고 힐링템으로 딱!  - 소리하라 투명 곰돌이 칼림바 
213 청주 직장인 취미 피아노학원 엠앤씨음악전문학원
214 인천 직장인 취미미술 클래스 (캘리그라피 & 수채화)
215 분당성인피아노학원 직장인취미 추천! 위드 피아노 방문후기
216 Permission to Dance  퍼미션 투 댄스 ㅡ BTS 방탄 안무 도전  / 직장인 성인 취미댄스 1:1 개인레슨  | 판교. 분당. 용인 댄스 개인레슨 전문 
217 [판교플라워클래스,판교직장인취미]플라워리스만드기 클래스
218 구래동 겨울시즌 성인미술수업/ 직장인 취미미술 아크릴화/소묘/수채화
219 살사댄스 건강한 직장인취미 뿜뿜 힐링하는 삶을 위한 준비! (이태원 용산 명동 왕십리)  
220 [직장인취미] 한국지식산업개발원 SNS마케팅전문가 1급 도전!!
221 직장인취미 수세미뜨기 도전
222 [모집] 석촌 잠실 직장인 기초 스페인어 스터디 / 알파벳부터 문법, 회화까지 취미로
223 직장인 인터넷 강의 클라스101 댓글 쓰고 클원 취미 지원금 받자
224 귀염진 테라조캔들 만들러 오신 직장인의 취미생활 미아사거리 캔들 공방 피어라 
225 DIY 직장인취미생활추천 귀여운 십자수 도안 액자 만들기
226 직장인 취미연기 <자아도취> 10월 독백 수업 이모저모!
227 직장인취미, 어떤 걸 시작하면 좋을지 고민된다면? 선택해요!
228 [정규클래스 | 드로잉] 용산 그림정규클래스 풍경화 그리기 오일파스텔화 / 용산 취미미술 직장인 성인 취미미술학원 미각그림공방
229 라탄바구니 정규반 직장인취미 서울 강서구 라탄 마곡동라탄공방 라비에벨라탄 
230 발린이 일기 1주차. 직장인 취미 발레
231 직장인 취미 성인피아노 레슨 가능한 피아노 리브레 여의도점 방문했어요!
232 직장인 취미, 취미 생활하면서 커리어 관리까지 가능하다고?
233 김포 발레 학원 직장인 취미로 OK!
234 

WebDriverException: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=96.0.4664.55)


In [ ]:
print('수집한 글 갯수: ', len(dict))

dict[0]

# 판다스화
result_df = pd.DataFrame.from_dict(dict, 'index')
result_df